In [3]:
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
from colorama import Fore, init
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import random
import sys

In [4]:
init() #Colorama init

random_wait = random.randint(2, 4)

#Red flags según artículo
flags_auto = ("conversable", "al dia", "detalles", "atrasada", "atrasado", "atrasados", "trizado", "trizada", "choques", "choque", "chocado", "deshuase", "desarme", "le falta", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "no transferible", "multa", "multas", "parte", "partes")

flags = flags_auto

close_locations = ["Valparaiso", "Viña del Mar"]

In [5]:
login = False #True da más detalles de los autos pero puede ser problemático, False da menos detalles pero es más robusto

# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 500000
max_price = 10000000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
min_km = 1000 #Menos de 1000 km suelen ser autos que omiten ceros
max_km = 300000
min_year = 1990
max_year = 2024
transmission = "automatic"
make = "chevrolet "
search_model = ""
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [6]:
# Leer html desde archivo

nombre_html = f"{make}_{transmission}.html"

with open(nombre_html, "r", encoding="utf-8") as file:
    html = file.read()

# Crear un objeto BeautifulSoup a partir del HTML

market_soup = soup(html, 'html.parser')

print(f"Se ha leído el HTML de la página desde {nombre_html}")

Se ha leído el HTML de la página desde chevrolet _automatic.html


In [7]:
# Idea: Extraer los elementos completos (cada cuadro) y recorrerlos buscando la información, en caso que no tenga todas las clases se ignora

# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_and_mileages_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
locations_and_mileages_list = [location_and_mileage.text.strip() for location_and_mileage in locations_and_mileages_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [17]:
for i in range(0, len(titles_div)):
    print(f"{i+1: <5}: {titles_list[i]: <50} - {titles_div[i]}")

1    : 2022 Kia morning                                   - <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2022 Kia morning</span>
2    : 2022 Chevrolet sail smart                          - <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2022 Chevrolet sail smart</span>
3    : 2017 Chevrolet cruze                               - <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2017 Chevrolet cruze</span>
4    : 2017 Chevrolet tracker                             - <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-webkit-box-orient: vertical; -webkit-line-clamp: 2; display: -webkit-box;">2017 Chevrolet tracker</span>
5    : 2012 Chevrolet captiva                             - <span class="x1lliihq x6ikm8r x10wlt62 x1n2onr6" style="-

In [8]:
#print elements to check

for i in range(0, len(titles_list)):
    print(f"{i+1}: {titles_list[i]} - {prices_list[i]} - {locations_and_mileages_list[i]} - {urls_list[i]}")

1: 2022 Kia morning - $3.400.000 - La Estrella, LI - https://web.facebook.com/marketplace/item/290372197416344/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2: 2022 Chevrolet sail smart - $3.800.000 - 8 mil millas - https://web.facebook.com/marketplace/item/311327251979392/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3: 2017 Chevrolet cruze - $10.000.000 - Santiago, RM - https://web.facebook.com/marketplace/item/956956796150563/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4: 2017 Chevrolet tracker - $8.500.000 - 54 mil km - https://web.facebook.com/marketplace/item/971173897698108/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
5: 2012 Chevrolet captiva - $6.000.000 - Peñaflor, RM - https://web.facebook.com/marketplace/item/661370269407166/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
6: 2013 Chevrolet traverse iii lt adw 3.6cc - $7.400.000 - 80 mil km - https://web.facebo

In [9]:
# locations_and_mileages_list

#print len of lists
print(f"Titles: {len(titles_list)}")
print(f"Prices: {len(prices_list)}")
print(f"Locations and mileages: {len(locations_and_mileages_list)/2}")
print(f"URLs: {len(urls_list)}")

# Stop program if lists are not the same length

if not len(titles_list) == len(prices_list) == len(locations_and_mileages_list)/2 == len(urls_list):
    print("Lists are not the same length. Exiting program.")
    #driver.quit()
    sys.exit()




Titles: 736
Prices: 737
Locations and mileages: 737.0
URLs: 737
Lists are not the same length. Exiting program.


SystemExit: 

C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+) km'
mileage_pattern_miles = r'(\d+) millas'

# Initialize an empty list to store adjusted mileage entries
locations_and_mileages_list2 = []

# Iterate through the original mileage entries
for item in locations_and_mileages_list:
    # Append the current mileage entry to the adjusted list
    locations_and_mileages_list2.append(item)	

    # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if item == '':
        locations_and_mileages_list2[-1] = item.replace('', '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")

    elif pattern.search(item) and len(locations_and_mileages_list2) >= 2 and pattern.search(locations_and_mileages_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        locations_and_mileages_list2.insert(-1, '0')
        print(f"Se encontró un espacio en blanco en el elemento {len(locations_and_mileages_list2)} y se rellenó con 0 km")  

    
    # Try to find a match for the "K km" format
    match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
    # Try to find a match for the "K miles" format
    match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])

    if "\xa0mil" in item: #Por esta parte se necesita que se busquen autos con mas de 1000 km
        locations_and_mileages_list2[-1] = item.replace("\xa0mil", "000")
        # Try to find a match for the "K km" format
        match_mileage_km = re.search(mileage_pattern_km, locations_and_mileages_list2[-1])
        
        # Try to find a match for the "K miles" format
        match_mileage_miles = re.search(mileage_pattern_miles, locations_and_mileages_list2[-1])
         
        
    # Check if either of the formats is found
    if match_mileage_km or match_mileage_miles:
        # If "K km" format is found, convert it to meters and append to the cleaned list
        if match_mileage_km:
            locations_and_mileages_list2[-1] = match_mileage_km.group(1)
        # If "K miles" format is found, convert it to meters and append to the cleaned list
        else:
            locations_and_mileages_list2[-1] = str((int(match_mileage_miles.group(1)) * 1609//1000))
            print(f"Se encontró una distancia en K millas {int(match_mileage_miles.group(1))}, se convirtió a {locations_and_mileages_list2[-1]} metros")

Se encontró una distancia en K millas 8000, se convirtió a 12872 metros
Se encontró una distancia en K millas 230000, se convirtió a 370070 metros
Se encontró una distancia en K millas 32000, se convirtió a 51488 metros
Se encontró una distancia en K millas 170000, se convirtió a 273530 metros
Se encontró una distancia en K millas 120000, se convirtió a 193080 metros
Se encontró un espacio en blanco en el elemento 196 y se rellenó con 0 km
Se encontró un espacio en blanco en el elemento 246 y se rellenó con 0 km
Se encontró una distancia en K millas 100000, se convirtió a 160900 metros
Se encontró una distancia en K millas 169000, se convirtió a 271921 metros
Se encontró un espacio en blanco en el elemento 360 y se rellenó con 0 km
Se encontró una distancia en K millas 10000, se convirtió a 16090 metros
Se encontró una distancia en K millas 222000, se convirtió a 357198 metros
Se encontró una distancia en K millas 61000, se convirtió a 98149 metros
Se encontró un espacio en blanco en e

In [ ]:
# Separate locations and mileages lists

# Initialize an empty list to store cleaned values
mileage_clean = []
locations_list = []

# Iterate through the locations_and_mileages_list2 in steps of 2 to separate locations and mileage
for i in range(0, len(locations_and_mileages_list2), 2):
    # Extract location
    location_parts = locations_and_mileages_list2[i].split(',')
    if len(location_parts) > 0:
        # Clean and decode the location to handle UTF-8 characters properly
        location = location_parts[0].strip().encode('utf-8').decode('utf-8')
        locations_list.append(location)   

    # Extract mileage
    mileage_parts = locations_and_mileages_list2[i + 1].split(' ')
    if len(mileage_parts) > 0:
        mileage = mileage_parts[0]
        try:
            mileage_clean.append(int(mileage))
        except:
            print(Fore.RED + locations_and_mileages_list2[i + 1] + Fore.RESET)

print("Locations:", locations_list)
print("Mileage:", mileage_clean)

Locations: ['La Estrella', 'Santiago', 'Peñaflor', 'Recoleta', 'Lampa', 'Buin', 'Santiago', 'Santiago', 'San Ramón', 'Curacaví', 'Santiago', 'Cerrillos', 'Estación Central', 'Santiago', 'Santiago', 'Lo Barnechea', 'Viña del Mar', 'Maipú', 'Maipú', 'Independencia', 'La Florida', 'San Bernardo', 'Las Condes', 'Quilicura', 'Santiago', 'Quilicura', 'Lampa', 'El Bosque', 'Lampa', 'Huechuraba', 'San Antonio', 'Las Condes', 'Santiago', 'El Bosque', 'La Cisterna', 'Los Andes', 'Casablanca', 'Algarrobo', 'Colina', 'Colina', 'Maipú', 'Santiago', 'Santiago', 'Las Condes', 'Maipú', 'Santiago', 'Puente Alto', 'Talca', 'Isla de Maipo', 'La Florida', 'Santiago', 'Quinta Normal', 'Rancagua', 'Maipú', 'La Florida', 'Las Condes', 'Talca', 'Curicó', 'San Miguel', 'Pedro Aguirre Cerda', 'Santiago', 'Viña del Mar', 'San Joaquín', 'Puente Alto', 'Santa Cruz', 'Talca', 'Santiago', 'Renca', 'Malloa', 'Santiago', 'Cerrillos', 'Maipú', 'Puente Alto', 'Chimbarongo', 'Rancagua', 'Providencia', 'Buin', 'Santiago',

In [ ]:
print(len(locations_list))
print(len(mileage_clean))

737
737


In [ ]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
index_to_delete = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    try: 
        precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
        # Agregar el precio limpio a la lista
        clean_prices_list.append(precio_limpio)
        
    except:
        try: 
            precio_limpio = int(precio.replace('.', '').replace('$', ''))
            # Agregar el precio limpio a la lista
            clean_prices_list.append(precio_limpio)
        except:
            #Delete element from the lists
            index = prices_list.index(precio)
            print(Fore.RED + f"Se encontró un precio no válido: {precio}, en la posición {index}" + Fore.RESET)
            print(urls_list[index])
            index_to_delete.append(index)

for index in sorted(index_to_delete, reverse=True):
    del prices_list[index]
    del titles_list[index]
    del locations_list[index]
    del mileage_clean[index]
    del urls_list[index]
        
        

In [ ]:
# print len of lists

print(f"Titles: {len(titles_list)}")
print(f"Clean Prices: {len(clean_prices_list)}")
print(f"Prices: {len(prices_list)}")
print(f"Locations: {len(locations_list)}")
print(f"Mileage: {len(mileage_clean)}")
print(f"URLs: {len(urls_list)}")

Titles: 736
Clean Prices: 737
Prices: 737
Locations: 737
Mileage: 737
URLs: 737


In [ ]:
# print every element from prices

for i in range(len(prices_list)):
    print(f"{i} {prices_list[i]} -> {clean_prices_list[i]}")

""" print(clean_prices_list[1:5])
print(prices_list[1:5]) """

: 

La parte de filtrado de resultados obtenidos es la más demorada de todas

In [ ]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(make)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    title_split = titles_list[i].split()

    # Check if the title contains the search make
    if re.search(patron, item):

        items_dict["Titel"] = item
        items_dict["Year"] = int(title_split[0])
        items_dict["Make"] = title_split[1]
        #items_dict["Model"] = title_split[2]
        items_dict["Model"] = ' '.join(title_split[2:])
        items_dict["Price"] = clean_prices_list[i]
        items_dict["Search Model"] = search_model
        items_dict["Transmission"] = transmission
        items_dict["Mileage"] = mileage_clean[i]
        items_dict["Location"] = locations_list[i]
        items_dict["Description"] = "hola mundo" #get_description(urls_list[i]) #La parte más lenta

        detected_flags = []            

        # Check for flags in the description
        description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas y CREO que para quitar acentos
        for flag in flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                if flag == 'detalles' and ('detalles esteticos' in description or 'detalles de pintura' in description or 'paqueños detalles' in description or 'detalles menores' in description or 'detalles minimos' in description):
                    # Si encuentra detalles menores se guarda en los flags como "detalles menores"
                    detected_flags.append('detalles menores')

                if flag == 'multa' and ('ninguna multa' in description or 'sin multa' in description):
                    continue
                elif flag == 'multas' and ('sin multas' in description or 'no tiene multas' in description or 'libre de multas' in description or 'ni multas' in description or 'no multas' in description):
                    continue
                elif flag == 'parte' and ('ningun parte' in description):
                    continue
                elif flag == 'partes' and ('sin partes' in description or 'no tiene partes' in description or 'libre de partes' in description or 'ni partes' in description or 'no partes' in description):
                    continue
                elif flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles'):
                    continue
                elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                    continue
                elif flag == 'choques' and ('sin choques' in description):
                    continue
                elif flag == 'chocado' and ('nunca chocado' in description):
                    continue
                else:
                    detected_flags.append(flag)

        if detected_flags:
            print(f"[{i}/{len(titles_list)}] Se encontraron los flags {detected_flags} en la siguiente descripción: {items_dict['Description']}")


        items_dict["Flags"] = detected_flags
        items_dict["URL"] = urls_list[i]

        items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")



Se obtuvieron 21 resultados utilizables
De una búsqueda de 23 resultados


In [ ]:
# Cerrar la sesión del navegador
#driver.quit()

In [ ]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
16,2008 Chevrolet spark,2008,Chevrolet,spark,2100000,,manual,126000,Casablanca,hola mundo,[],https://web.facebook.com/marketplace/item/1150110439236164/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
17,2011 Chevrolet spark,2011,Chevrolet,spark,2200000,,manual,126000,San Bernardo,hola mundo,[],https://web.facebook.com/marketplace/item/1300611287273374/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
18,2008 Chevrolet spark lt,2008,Chevrolet,spark lt,2100000,,manual,108000,Quilicura,hola mundo,[],https://web.facebook.com/marketplace/item/4209172435975706/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
19,2014 Chevrolet sail,2014,Chevrolet,sail,3200000,,manual,120000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/3775383182785425/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
20,2011 Chevrolet spark,2011,Chevrolet,spark,2650000,,manual,150000,Maipú,hola mundo,[],https://web.facebook.com/marketplace/item/1796273880871509/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [ ]:
items_df

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,Flags,URL
0,2004 Chevrolet astra,2004,Chevrolet,astra,2900000,,manual,10000,Curacaví,hola mundo,[],https://web.facebook.com/marketplace/item/4566985566860382/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1,2019 Chevrolet prisma lt,2019,Chevrolet,prisma lt,5500000,,manual,1000,Quinta Normal,hola mundo,[],https://web.facebook.com/marketplace/item/1118496896134802/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2,2020 Chevrolet onix nb ls 1.2,2020,Chevrolet,onix nb ls 1.2,6500000,,manual,100000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/948946446815603/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3,2018 Chevrolet prisma,2018,Chevrolet,prisma,2800000,,manual,180000,Lampa,hola mundo,[],https://web.facebook.com/marketplace/item/459293053128222/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4,2005 Chevrolet optra 1.6,2005,Chevrolet,optra 1.6,1300000,,manual,245000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/703458958395068/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
5,2018 Chevrolet sail chevrolet full,2018,Chevrolet,sail chevrolet full,5300000,,manual,134000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/1105575083996440/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
6,2013 Chevrolet spark,2013,Chevrolet,spark,2500000,,manual,160000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/7152812338160762/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
7,2018 Chevrolet prisma,2018,Chevrolet,prisma,5400000,,manual,72000,Santiago,hola mundo,[],https://web.facebook.com/marketplace/item/756439106461476/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
8,2016 Chevrolet sail,2016,Chevrolet,sail,4600000,,manual,76000,Cartagena,hola mundo,[],https://web.facebook.com/marketplace/item/909353294006466/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
9,2019 Chevrolet tracker,2019,Chevrolet,tracker,8790000,,manual,80000,Estación Central,hola mundo,[],https://web.facebook.com/marketplace/item/1175744023419363/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [ ]:
#Codigo para modificar la columna de precios

csv_name = f"{make.upper()}_{transmission.upper()}.csv"

# Cargar el archivo CSV en un DataFrame
items_cargados_df = pd.read_csv(csv_name)

# Modificar la columna deseada
items_cargados_df['Price'] = items_df['Price'] 

# Guardar el DataFrame modificado en el mismo archivo CSV
items_df.to_csv(csv_name, index=False)

print(f"Se modificó y guardó el DataFrame en el archivo {csv_name}")

Se modificó y guardó el DataFrame en el archivo CHEVROLET _MANUAL.csv
